In [1]:
from google.colab import drive
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import requests
from PIL import Image
from io import BytesIO
import ast
from sklearn import preprocessing
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
drive.mount('/content/drive')
loc = "/content/drive/MyDrive/dataset/"

Mounted at /content/drive


In [3]:
# Load the data
df = pd.read_csv(loc+'Assign2_Data.csv')
# df['ID'] = range(1, len(df) + 1)
df2 = df.copy()
df2.rename(columns={'Unnamed: 0':'IDs'}, inplace=True )

In [7]:
loc2 = "/content/drive/MyDrive/Images/"

In [43]:
def download(url,image_id,loc2,i):
  try:
      # Send a HTTP request to the URL of the image
      response = requests.get(url)

      # Open the URL image as a raw image
      img = Image.open(BytesIO(response.content))

      img.save(loc2+f"{image_id}_{i+1}.jpg")
  except Exception as e:
    print(f"Error processing URL {url}: {e}")

In [ ]:
dfi = df.copy()
dfi.rename(columns={'Unnamed: 0':'IDs'}, inplace=True)
dfi.dropna()

In [ ]:
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image

model = VGG16(weights='imagenet', include_top=False)

for url_list in dfi['Image']:
  url_list = ast.literal_eval(url_list)
  r = dfi[dfi['Image'].apply(lambda x: url_list[0] in x)]
  image_id = r['IDs'].values[0]
  for i in range(len(url_list)):
    url = url_list[i]
    print(url)
    img = download(url,image_id,loc2,i)


In [53]:
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
import os

In [70]:
image_files = [f for f in os.listdir(loc2) if f.endswith('.jpg')]
len(image_files)

1624

In [ ]:
features_dict = {}
xs = []
for i in image_files[:1000]:
  s = i
  s = s.split('_')
  id = int(s[0])
  img = image.load_img(loc2+i,target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)
  x = preprocess_input(x)
  xs.append(x)
  features = model.predict(x)
  if id in features_dict:
    # If the ID exists, append the new features to the existing list
    features_dict[id].append(features)
  else:
    # If the ID does not exist, create a new list with the features
    features_dict[id] = [features]
# img = img.resize((224, 224))
# x = image.img_to_array(img)
# x = image.expand_dims(x,axis=0)
# x = preprocess_input(x)
# features = model.predict(x)
# features_dict[url] =

In [ ]:
for i in image_files[1000:1624]:
  s = i
  s = s.split('_')
  id = int(s[0])
  img = image.load_img(loc2+i,target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)
  x = preprocess_input(x)
  xs.append(x)
  features = model.predict(x)
  if id in features_dict:
    # If the ID exists, append the new features to the existing list
    features_dict[id].append(features)
  else:
    # If the ID does not exist, create a new list with the features
    features_dict[id] = [features]

In [75]:
feature_df = pd.DataFrame(list(features_dict.items()), columns=['ID', 'Features'])
feature_df.head()

,ID,Features
0,3452,[[[[[39.933197 0. 0. ... 0. ...
1,1205,[[[[[ 0. 0. 0. ... 0. ...
2,1708,[[[[[ 0. 0. 0. ... 0. ...
3,2078,[[[[[ 0. 0. 49.539814 ... 0. ...
4,801,[[[[[ 0. 0. 7.23981 ... 0. ...


In [76]:
with open(loc+'features.pkl', 'wb') as f:
    pickle.dump(feature_df, f)

In [4]:
with open(loc+'features.pkl', 'rb') as f:
    feature_df = pickle.load(f)
feature_df['Features']

0      [[[[[39.933197   0.         0.        ...  0. ...
1      [[[[[ 0.         0.         0.        ...  0. ...
2      [[[[[ 0.        0.        0.       ...  0.    ...
3      [[[[[ 0.         0.        49.539814  ...  0. ...
4      [[[[[ 0.        0.        7.23981  ...  0.    ...
                             ...                        
975    [[[[[ 0.         0.         0.        ...  0. ...
976    [[[[[ 0.        0.        0.       ...  0.    ...
977    [[[[[20.933159  0.        0.       ...  0.    ...
978    [[[[[ 0.         0.         0.        ...  0. ...
979    [[[[[0.        0.        0.        ... 0.     ...
Name: Features, Length: 980, dtype: object

In [5]:
# from sklearn.preprocessing import StandardScaler
Normalized_features = []
# scaler = StandardScaler()
for i in feature_df['Features']:
  # print(type(i))
  temp = []
  for f in i:
    f = f.reshape(f.shape[0], -1)
    nf = preprocessing.normalize(f)
    temp.append(nf)
  Normalized_features.append(temp)
feature_df['Final Features'] = Normalized_features
feature_df

,ID,Features,Final Features
0,3452,[[[[[39.933197 0. 0. ... 0. ...,"[[[0.019927489, 0.0, 0.0, 0.0, 0.0, 0.0, 0.022..."
1,1205,[[[[[ 0. 0. 0. ... 0. ...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,1708,[[[[[ 0. 0. 0. ... 0. ...,"[[[0.0, 0.0, 0.0, 0.0, 0.0067598196, 0.0, 0.0,..."
3,2078,[[[[[ 0. 0. 49.539814 ... 0. ...,"[[[0.0, 0.0, 0.02513755, 0.00084237504, 0.0041..."
4,801,[[[[[ 0. 0. 7.23981 ... 0. ...,"[[[0.0, 0.0, 0.004710846, 0.0, 0.0048490968, 0..."
...,...,...,...
975,1606,[[[[[ 0. 0. 0. ... 0. ...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000384..."
976,1734,[[[[[ 0. 0. 0. ... 0. ...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005269..."
977,2112,[[[[[20.933159 0. 0. ... 0. ...,"[[[0.009752154, 0.0, 0.0, 0.0, 0.0, 0.00018292..."
978,2401,[[[[[ 0. 0. 0. ... 0. ...,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


In [18]:
feature_df2 = feature_df.copy()
feature_df2 = feature_df2.drop('Features',axis=1)
merged_df2 = pd.merge(df2, feature_df2[['ID','Final Features']], left_on='IDs', right_on='ID', how='left')
merged_df2['Final Features'].fillna(value='empty', inplace=True)
empty_lists_df = merged_df2[merged_df2['Final Features'].apply(lambda x: x and x == 'empty')]

# Now empty_lists_df contains rows where 'adjacent_column' is an empty list
print(empty_lists_df)

      IDs                                              Image  \
67   2235  ['https://images-na.ssl-images-amazon.com/imag...   
110  3317  ['https://images-na.ssl-images-amazon.com/imag...   
222  1978  ['https://images-na.ssl-images-amazon.com/imag...   
237  2416  ['https://images-na.ssl-images-amazon.com/imag...   
282  2400  ['https://images-na.ssl-images-amazon.com/imag...   
481  2733  ['https://images-na.ssl-images-amazon.com/imag...   
494     4  ['https://images-na.ssl-images-amazon.com/imag...   
523  2912  ['https://images-na.ssl-images-amazon.com/imag...   
600  3444  ['https://images-na.ssl-images-amazon.com/imag...   
605  1576  ['https://images-na.ssl-images-amazon.com/imag...   
611   107  ['https://images-na.ssl-images-amazon.com/imag...   
701  2265  ['https://images-na.ssl-images-amazon.com/imag...   
746  1925  ['https://images-na.ssl-images-amazon.com/imag...   
754   725  ['https://images-na.ssl-images-amazon.com/imag...   
806  3161  ['https://images-na.ssl-image

In [5]:
# Lower-casing
df['Review Text'] = df['Review Text'].str.lower()

# Fill missing values with empty strings
df['Review Text'] = df['Review Text'].fillna('')

# Ensure that all data in the 'Review Text' column are strings
df['Review Text'] = df['Review Text'].astype(str)

# Tokenization
df['Review Text'] = df['Review Text'].apply(nltk.word_tokenize)

# Removing punctuations
punc = string.punctuation
df['Review Text'] = df['Review Text'].apply(lambda x: [word for word in x if word not in punc])

# Stop Word Removal
stop_words = set(stopwords.words('english'))
df['Review Text'] = df['Review Text'].apply(lambda x: [word for word in x if word not in stop_words])

# Stemming
stemmer = PorterStemmer()
df['Review Text'] = df['Review Text'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x]))

# Lemmatization
lemmatizer = WordNetLemmatizer()
df['Review Text'] = df['Review Text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(x)]))

# with open('processed_reviews.pkl', 'wb') as f:
#     pickle.dump(df, f)

In [ ]:
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(df['Review Text'])
# tfidf_scores = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
# tfidf_scores

In [ ]:
df2.head(1)

In [8]:
def com_tf(words_set,corpus):
  n_docs = len(corpus)         #·Number of documents in the corpus
  n_words_set = len(words_set) #·Number of unique words in the
  words_set = list(words_set)
  df_tf = pd.DataFrame(np.zeros((n_docs, n_words_set)), columns=words_set)

  # Compute Term Frequency (TF)
  for i in range(n_docs):
      words = corpus[i].split(' ') # Words in the document
      for w in words:
          df_tf[w][i] = df_tf[w][i] + (1 / len(words))

  return df_tf

In [9]:
def com_idf(words_set,corpus):
  n_docs = len(corpus)
  n_words_set = len(words_set)

  idf = {}

  for w in words_set:
      k = 0

      for i in range(n_docs):
          if w in corpus[i].split():
              k += 1

      if k==0:
        np.log10(n_docs / (k+1))
      else:
        idf[w] =  np.log10(n_docs / k)
  return idf

In [10]:
def com_tf_idf(df_tf, corpus, words_set, idf):
  n_docs = len(corpus)
  df_tf_idf = df_tf.copy()

  for w in words_set:
    if w!='':
      for i in range(n_docs):
        df_tf_idf[w][i] = df_tf[w][i] * idf[w]

  return df_tf_idf

In [11]:
corpus = list(df['Review Text'])

words_set = set()

for doc in corpus:
    words = doc.split(' ')
    words_set = words_set.union(set(words))

In [12]:
tf = com_tf(words_set, corpus)
idf = com_idf(words_set, corpus)

In [13]:
tfidf = com_tf_idf(tf, corpus, words_set, idf)

In [49]:
with open(loc+'tfidf_scores.pkl', 'wb') as f:
    pickle.dump(tfidf, f)

In [19]:
with open(loc+'tfidf_scores.pkl', 'rb') as f:
    tfidf = pickle.load(f)
tfidf

,IDs,,event,325,slowli,mesh,profil,siii,nostalgia,thier,...,6000,combin,banger,sweeten,favor,alright,recip,greedi,suggest,dremmel
0,3452,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,1882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,1547,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
df3 = df2.dropna()
df3

,IDs,Image,Review Text
0,3452,['https://images-na.ssl-images-amazon.com/imag...,Loving these vintage springs on my vintage str...
1,1205,['https://images-na.ssl-images-amazon.com/imag...,Works great as a guitar bench mat. Not rugged ...
2,1708,['https://images-na.ssl-images-amazon.com/imag...,We use these for everything from our acoustic ...
3,2078,['https://images-na.ssl-images-amazon.com/imag...,Great price and good quality. It didn't quite...
4,801,['https://images-na.ssl-images-amazon.com/imag...,I bought this bass to split time as my primary...
...,...,...,...
995,1265,['https://images-na.ssl-images-amazon.com/imag...,Extremely impressed with this kit.
996,1882,['https://images-na.ssl-images-amazon.com/imag...,This is a great stereo reverb with plenty of c...
997,1547,['https://images-na.ssl-images-amazon.com/imag...,I really like the simplicity of this bridge. I...
998,1004,['https://images-na.ssl-images-amazon.com/imag...,"Great Product, but there is no warranty in the..."


In [21]:
inp1 = input('Image URL: \n')
inp = input('Review: \n')
row = df2[df2['Review Text'] == inp]

review_id = row['IDs'].values[0]
print(review_id)

Image URL: 
https://images-na.ssl-images-amazon.com/images/I/81PTFWPHbxL._SY88.jpg
Review: 
Perfect for my EC-1000S EMG. Been using D'addario for years and they never disappoint me. Even on a shorter scale guitar there is enough tension to keep them tight.
199


In [22]:
def find(inp1):
  r = df3[df3['Image'].apply(lambda x: inp1 in x)]
  image_id = r['IDs'].values[0]
  index = df3[df3['IDs'] == image_id].index[0]
  list_in_row = df3.loc[index, 'Image']
  print(list_in_row)
  inde = 0
  for i in range(len(list_in_row)):
    if list_in_row[i]==inp1:
      inde = i
      break
  return image_id, inde

In [23]:
def find_features(inp1,feature_df2):
  id, ind = find(inp1)
  roww = feature_df2[feature_df2['ID'] == id].index[0]
  img_vector = feature_df2.loc[roww,'Final Features']
  print(img_vector)
  img_vector = img_vector[ind]
  return img_vector

In [24]:
sim_dict = {}
img_vec = find_features(inp1,feature_df2)
len(img_vec)

['https://images-na.ssl-images-amazon.com/images/I/81PTFWPHbxL._SY88.jpg']
[array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)]


1

In [ ]:
for i,k in zip(feature_df2['Final Features'],feature_df2['ID']):
  id2, ind2 = find(inp1)
  if id2!=k:
    row2 = feature_df2[feature_df2['ID'] == k].index[0]
    img_vec2 = feature_df2.loc[row2,'Final Features']
    for j in img_vec2:
      cos_sim = cosine_similarity(img_vec, j)[0][0]
      if k in sim_dict:
        # If the ID exists, append the new features to the existing list
        sim_dict[k].append(cos_sim)
      else:
        # If the ID does not exist, create a new list with the features
        sim_dict[k] = [cos_sim]

In [26]:
df_similar_img = pd.DataFrame(sim_dict.items(), columns=['ID', 'Similarity'])
df_similar_img['Similarity'] = df_similar_img['Similarity'].apply(np.mean)
df_sorted = df_similar_img.sort_values(by='Similarity', ascending=False)
top_3_ids = df_sorted.head(3)

In [43]:
len(sim_dict)

979

In [27]:
top_3_ids['Review'] = top_3_ids['ID'].apply(lambda id: df3[df3['IDs'] == id]['Review Text'].values[0])
top_3_ids['Image'] = top_3_ids['ID'].apply(lambda id: df3[df3['IDs'] == id]['Image'].values[0])
top_3_ids

<ipython-input-27-16556b910ed5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_3_ids['Review'] = top_3_ids['ID'].apply(lambda id: df3[df3['IDs'] == id]['Review Text'].values[0])
<ipython-input-27-16556b910ed5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_3_ids['Image'] = top_3_ids['ID'].apply(lambda id: df3[df3['IDs'] == id]['Image'].values[0])


,ID,Similarity,Review,Image
767,2528,0.384541,Good,['https://images-na.ssl-images-amazon.com/imag...
566,674,0.340109,Very awesome strap! It looks exponentially coo...,['https://images-na.ssl-images-amazon.com/imag...
23,1572,0.340005,I wanted to customize my fake plastic rock ban...,['https://images-na.ssl-images-amazon.com/imag...


In [44]:
row = tfidf[tfidf['IDs'] == review_id]
vector = tfidf.loc[tfidf['IDs'] == review_id]
vector = vector.drop('IDs', axis=1)

similar_review = {}
i = 0
for review in list(df3['Review Text']):
  # i+=1
  # print(i)
  if review!='':
    row2 = df2[df2['Review Text'] == review]
    id2 = row2['IDs'].values[0]
  # print(id2)
    if review!=inp and id2!=review_id:
      vector2 = tfidf.loc[tfidf['IDs'] == id2]
      # print(vector2)
      vector2 = vector2.drop('IDs', axis=1)
      similarity = cosine_similarity(vector, vector2)[0][0]
      similar_review[review] = similarity

df_similar1 = pd.DataFrame(similar_review.items(), columns=['Review', 'Similarity1'])

df_similar1['ID'] = df_similar1['Review'].apply(lambda review: df2[df2['Review Text'] == review]['IDs'].values[0])
df_similar1['Image'] = df_similar1['Review'].apply(lambda review: df2[df2['Review Text'] == review]['Image'].values[0])

sim = sorted(similar_review.items(), key=lambda x: x[1], reverse=True)[:3]

In [42]:
print(len(similar_review))

995


In [29]:
df_similar = pd.DataFrame(sim, columns=['Review', 'Similarity'])

df_similar['ID'] = df_similar['Review'].apply(lambda review: df2[df2['Review Text'] == review]['IDs'].values[0])
df_similar['Image'] = df_similar['Review'].apply(lambda review: df2[df2['Review Text'] == review]['Image'].values[0])
df_similar

,Review,Similarity,ID,Image
0,a little bit shorter than expected but pretty ...,0.235813,1540,['https://images-na.ssl-images-amazon.com/imag...
1,"Great Quality, adjustable tension. Well made.",0.213311,2939,['https://images-na.ssl-images-amazon.com/imag...
2,"I love D'Addario strings, however, this partic...",0.190251,203,['https://images-na.ssl-images-amazon.com/imag...


In [53]:
new_dff = pd.merge(df_similar1,df_similar_img,on='ID',how='left')
new_dff.fillna(0,inplace=True)
new_dff

,Review,Similarity1,ID,Image,Similarity
0,Loving these vintage springs on my vintage str...,0.083010,3452,['https://images-na.ssl-images-amazon.com/imag...,0.142199
1,Works great as a guitar bench mat. Not rugged ...,0.024764,1205,['https://images-na.ssl-images-amazon.com/imag...,0.128396
2,We use these for everything from our acoustic ...,0.040868,1708,['https://images-na.ssl-images-amazon.com/imag...,0.151013
3,Great price and good quality. It didn't quite...,0.038976,2078,['https://images-na.ssl-images-amazon.com/imag...,0.175147
4,I bought this bass to split time as my primary...,0.000000,801,['https://images-na.ssl-images-amazon.com/imag...,0.276184
...,...,...,...,...,...
990,I installed these myself using the Elenco Elec...,0.001808,212,['https://images-na.ssl-images-amazon.com/imag...,0.120481
991,Extremely impressed with this kit.,0.000000,1265,['https://images-na.ssl-images-amazon.com/imag...,0.261168
992,This is a great stereo reverb with plenty of c...,0.000000,1882,['https://images-na.ssl-images-amazon.com/imag...,0.150714
993,I really like the simplicity of this bridge. I...,0.014192,1547,['https://images-na.ssl-images-amazon.com/imag...,0.146175


In [30]:
def image_similarity(url,k):
  rows = feature_df2[feature_df2['ID'] == k].index[0]
  img_vector = feature_df2.loc[rows,'Final Features']
  s = 0
  for j in img_vector:
      cos_img = cosine_similarity(img_vec, j)[0][0]
      s += cos_img
  return s/len(img_vector)

In [31]:
def text_similarity(review):
  if review!='':
    row2 = df2[df2['Review Text'] == review].index[0]
    id2 = df2.loc[row2,'IDs']
    if review!=inp and id2!=review_id:
      vector2 = tfidf.loc[tfidf['IDs'] == id2]
      vector2 = vector2.drop('IDs', axis=1)
      similarity = cosine_similarity(vector, vector2)[0][0]
      return similarity

In [57]:
def composite_scores(new_dff):
  new_dff['mean_similarity'] = (new_dff['Similarity1'] + new_dff['Similarity']) / 2
  top_3_rows = new_dff.nlargest(6, 'mean_similarity')
  return top_3_rows

In [111]:
comp_scores_dict = {}

In [112]:
print('USING IMAGE RETRIEVAL\n')
for index, row in top_3_ids.iterrows():
    print(f"Image URL: {row['Image']}")
    print(f"Review: {row['Review']}")
    print(f"Cosine similarity of image: {row['Similarity']:.4f}")
    print(f"Cosine similarity of text: {text_similarity(row['Review']):.4f}")
    print(f"Composite similarity score: {(row['Similarity']+text_similarity(row['Review']))/2:.4f}")
    comp_scores_dict[row['ID']] = [row['Image'],row['Review'],row['Similarity'],text_similarity(row['Review']),(row['Similarity']+text_similarity(row['Review']))/2]
    print("\n")

USING IMAGE RETRIEVAL

Image URL: ['https://images-na.ssl-images-amazon.com/images/I/81K9NmC4xVL._SY88.jpg']
Review: Good
Cosine similarity of image: 0.3845
Cosine similarity of text: 0.0000
Composite similarity score: 0.1923


Image URL: ['https://images-na.ssl-images-amazon.com/images/I/61tCh5LlsWL._SY88.jpg']
Review: Very awesome strap! It looks exponentially cool! Leather ends are supple enough to install to strap buttons easily. Strap itself is really soft, so it doesn't irritate your shoulders and neck while you're playing your guitar. It adjusts pretty easy to your desired length. Buckle is plastic. You may or may not have an issue with that. I personally didn't. It looks amazing, and that's putting it kindly. I've gotten many compliments on it already. Definitely worth the money. As a back story, I was born on the year of the Dragon, so it's only fitting I bought this strap.
Cosine similarity of image: 0.3401
Cosine similarity of text: 0.0300
Composite similarity score: 0.1851


In [114]:
print('USING TEXT RETRIEVAL\n')
for index, row in df_similar.iterrows():
    print(f"Image URL: {row['Image']}")
    cos_img = image_similarity(row['Image'],row['ID'])
    print(f"Review: {row['Review']}")
    print(f"Cosine similarity of images: {cos_img:.4f}")
    print(f"Cosine similarity of text: {row['Similarity']:.4f}")
    print(f"Composite similarity score: {(row['Similarity']+cos_img)/2:.4f}")
    comp_scores_dict[row['ID']] = [row['Image'],row['Review'],cos_img,row['Similarity'],(row['Similarity']+cos_img)/2]
    print("\n")

USING TEXT RETRIEVAL

Image URL: ['https://images-na.ssl-images-amazon.com/images/I/81UcQc6a-HL._SY88.jpg']
Review: a little bit shorter than expected but pretty good
Cosine similarity of images: 0.1510
Cosine similarity of text: 0.2358
Composite similarity score: 0.1934


Image URL: ['https://images-na.ssl-images-amazon.com/images/I/81U3GJsTjNL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71TDWb-prbL._SY88.jpg']
Review: Great Quality, adjustable tension. Well made.
Cosine similarity of images: 0.2459
Cosine similarity of text: 0.2133
Composite similarity score: 0.2296


Image URL: ['https://images-na.ssl-images-amazon.com/images/I/61n0yI7eC7L._SY88.jpg']
Review: I love D'Addario strings, however, this particular set when installed on my 28.625" scale 8-string - the top .74 string was not wound far enough to reach the nut. This left me with roughly 1/3" of the string that does not match (Pic). I'm honestly not sure if this is an isolated incident or if all packs will y

In [118]:
print('USING COMPOSITE SCORES FOR RETRIEVAL\n')
last = sorted(comp_scores_dict.items(),key=lambda x: x[1][-1],reverse=True)
for i in last:
    print(f"Image URL: {i[1][0]}")
    print(f"Review: {i[1][1]}")
    print(f"Cosine similarity of images: {i[1][2]:.4f}")
    print(f"Cosine similarity of text: {i[1][3]:.4f}")
    print(f"Composite similarity score: {i[1][4]:.4f}")
    print("\n")

USING COMPOSITE SCORES FOR RETRIEVAL

Image URL: ['https://images-na.ssl-images-amazon.com/images/I/81U3GJsTjNL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71TDWb-prbL._SY88.jpg']
Review: Great Quality, adjustable tension. Well made.
Cosine similarity of images: 0.2459
Cosine similarity of text: 0.2133
Composite similarity score: 0.2296


Image URL: ['https://images-na.ssl-images-amazon.com/images/I/81UcQc6a-HL._SY88.jpg']
Review: a little bit shorter than expected but pretty good
Cosine similarity of images: 0.1510
Cosine similarity of text: 0.2358
Composite similarity score: 0.1934


Image URL: ['https://images-na.ssl-images-amazon.com/images/I/81K9NmC4xVL._SY88.jpg']
Review: Good
Cosine similarity of images: 0.3845
Cosine similarity of text: 0.0000
Composite similarity score: 0.1923


Image URL: ['https://images-na.ssl-images-amazon.com/images/I/61tCh5LlsWL._SY88.jpg']
Review: Very awesome strap! It looks exponentially cool! Leather ends are supple enough to instal